In [1]:
import sys
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras import callbacks
import time

Using TensorFlow backend.


In [2]:

DEV = False
argvs = sys.argv
argc = len(argvs)

if argc > 1 and (argvs[1] == "--development" or argvs[1] == "-d"):
  DEV = True

if DEV:
  epochs = 2
else:
  epochs = 20

In [3]:
train_data_path = 'data1/train'
validation_data_path = 'data1/test'

In [4]:
img_width, img_height = 150, 150
batch_size = 32
samples_per_epoch = 1000
validation_steps = 300
nb_filters1 = 32
nb_filters2 = 64
conv1_size = 3
conv2_size = 2
pool_size = 2
classes_num = 4
lr = 0.0004

In [5]:
model = Sequential()
model.add(Convolution2D(nb_filters1, conv1_size, conv1_size, border_mode ="same", input_shape=(img_width, img_height, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Convolution2D(nb_filters2, conv2_size, conv2_size, border_mode ="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size), dim_ordering='th'))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(classes_num, activation='softmax'))


/home/ubuntu/anaconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(150, 150,..., padding="same")`
  
/home/ubuntu/anaconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (2, 2), padding="same")`
  
/home/ubuntu/anaconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`
  


In [6]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=lr),
              metrics=['accuracy'])

In [7]:

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 14588 images belonging to 4 classes.
Found 249 images belonging to 4 classes.


In [8]:

model.fit_generator(
    train_generator,
    samples_per_epoch=samples_per_epoch,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_steps)

/home/ubuntu/anaconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=20, validation_data=<keras_pre..., validation_steps=300, steps_per_epoch=31)`
  


Epoch 1/20
31/31 [==============================] - 63s 2s/step - loss: 3.0866 - acc: 0.5222 - val_loss: 1.1760 - val_acc: 0.5749
Epoch 2/20
31/31 [==============================] - 116s 4s/step - loss: 0.3072 - acc: 0.8992 - val_loss: 0.2450 - val_acc: 0.9596
Epoch 3/20
31/31 [==============================] - 55s 2s/step - loss: 0.1671 - acc: 0.9597 - val_loss: 0.4940 - val_acc: 0.9356
Epoch 4/20
31/31 [==============================] - 46s 1s/step - loss: 0.1223 - acc: 0.9627 - val_loss: 0.7204 - val_acc: 0.9041
Epoch 5/20
31/31 [==============================] - 47s 2s/step - loss: 0.1262 - acc: 0.9688 - val_loss: 0.2595 - val_acc: 0.9479
Epoch 6/20
31/31 [==============================] - 46s 1s/step - loss: 0.0708 - acc: 0.9768 - val_loss: 0.6474 - val_acc: 0.8353
Epoch 7/20
31/31 [==============================] - 45s 1s/step - loss: 0.0892 - acc: 0.9829 - val_loss: 0.4958 - val_acc: 0.9358
Epoch 8/20
31/31 [==============================] - 45s 1s/step - loss: 0.1031 - acc: 0.9

In [9]:
target_dir = './models/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
model.save('./models/model3.h5')
model.save_weights('./models/weights3.h5')

In [10]:
#Define Path
model_path = './models/model.h5'
model_weights_path = './models/weights.h5'
test_path = 'data/al_test'


#Define image parameters
img_width, img_height = 150, 150

#Prediction Function
def predict(file):
  x = load_img(file, target_size=(img_width,img_height))
  x = img_to_array(x)
  x = np.expand_dims(x, axis=0)
  array = model.predict(x)
  result = array[0]
  #print(result)
  answer = np.argmax(result)
  if answer == 1:
    print("Predicted: chair")
  elif answer == 0:
    print("Predicted: Motorbikes")
  elif answer == 2:
    print("Predicted: soccer_ball")

  return answer

#Walk the directory for every image
for i, ret in enumerate(os.walk(test_path)):
  for i, filename in enumerate(ret[2]):
    if filename.startswith("."):
        
      continue
    
    print(ret[0] + '/' + filename)
    result = predict(ret[0] + '/' + filename)
    print(" ")
